# Collect images

In [1]:
# import os

# import cv2


# DATA_DIR = './data'
# if not os.path.exists(DATA_DIR):
#     os.makedirs(DATA_DIR)

# number_of_classes = 3
# dataset_size = 100

# cap = cv2.VideoCapture(0)
# for j in range(number_of_classes):
#     if not os.path.exists(os.path.join(DATA_DIR, str(j))):
#         os.makedirs(os.path.join(DATA_DIR, str(j)))

#     print('Collecting data for class {}'.format(j))

#     done = False
#     while True:
#         ret, frame = cap.read()
#         cv2.putText(frame, 'Ready? Press "Q" ! :)', (100, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3,
#                     cv2.LINE_AA)
#         cv2.imshow('frame', frame)
#         if cv2.waitKey(25) == ord('q'):
#             break

#     counter = 0
#     while counter < dataset_size:
#         ret, frame = cap.read()
#         cv2.imshow('frame', frame)
#         cv2.waitKey(25)
#         cv2.imwrite(os.path.join(DATA_DIR, str(j), '{}.jpg'.format(counter)), frame)
#         if cv2.waitKey(25) == ord('q'):
#             break

#         counter += 1
    

# cap.release()
# cv2.destroyAllWindows()
# cv2.waitKey(1)

# Create Dataset

In [2]:
import os
import pickle
import mediapipe as mp
import cv2
import matplotlib.pyplot as plt

In [3]:
# # data_dir='./data'

# # mp_hands=mp.solutions.hands
# # mp_drawing=mp.solutions.drawing_utils
# # mp_drawing_styles=mp.solutions.drawing_styles

# # hands=mp_hands.Hands(static_image_mode=True,min_detection_confidence=0.3)

# # data=[]
# # label=[]

# for dir_ in os.listdir(data_dir):
#     if dir_=='.gitignore' or dir_=='.DS_Store':
#         continue
#     for img_path in os.listdir(os.path.join(data_dir,dir_)):
#         data_img=[]
#         img=cv2.imread(os.path.join(data_dir,dir_,img_path))
#         img_rgb=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        
#         results=hands.process(img_rgb)
        
# #         if results.multi_hand_landmarks:
# #             for hand_landmarks in results.multi_hand_landmarks:
# #                 mp_drawing.draw_landmarks(
# #                     img_rgb, #image to draw
# #                     hand_landmarks, #model output
# #                     mp_hands.HAND_CONNECTIONS,
# #                     mp_drawing_styles.get_default_hand_landmarks_style(),
# #                     mp_drawing_styles.get_default_hand_connections_style())
#         if results.multi_hand_landmarks:
#             for hand_landmarks in results.multi_hand_landmarks:
#                 for i in range(len(hand_landmarks.landmark)):
#                     x=hand_landmarks.landmark[i].x
#                     y=hand_landmarks.landmark[i].y
#                     data_img.append(x)
#                     data_img.append(y)
#             data.append(data_img)
#             label.append(dir_)
# #                     print(i)
# #                     print(hand_landmarks.landmark[i])
                
# # #         plt.figure()
# # #         plt.imshow(img_rgb)
# # # plt.show()

In [ ]:
data_dir='./data'

mp_hands=mp.solutions.hands
mp_drawing=mp.solutions.drawing_utils
mp_drawing_styles=mp.solutions.drawing_styles

hands=mp_hands.Hands(static_image_mode=True,min_detection_confidence=0.3)

data=[]
label=[]

for dir_ in os.listdir(data_dir):
    if dir_=='.gitignore' or dir_=='.DS_Store':
        continue
    for img_path in os.listdir(os.path.join(data_dir,dir_)):
        data_img=[]
        img=cv2.imread(os.path.join(data_dir,dir_,img_path))
        img_rgb=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        
        results=hands.process(img_rgb)
        
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                for i in range(len(hand_landmarks.landmark)):
                    x=hand_landmarks.landmark[i].x
                    y=hand_landmarks.landmark[i].y
                    data_img.append(x)
                    data_img.append(y)
            data.append(data_img)
            label.append(dir_)


                

I0000 00:00:1706218957.410417       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 86), renderer: Apple M2 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [ ]:
f=open('data.pickle','wb')
pickle.dump({'data':data,'label':label},f)
f.close()

# Train Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import numpy as np

In [ ]:
data_dict=pickle.load(open('data.pickle','rb'))

data=np.asarray(data_dict['data'])
label=np.asarray(data_dict['label'])

x_train,x_test,y_train,y_test=train_test_split(data,label,test_size=0.2,shuffle=True,stratify=label)

"""
stratify is used to make sure that equal proportion of data is present in both train and test sets, i.e., if 1/3rd 
of data in train set is from folder 0 then 1/3rd of data in test set is also from folder 0.
""" 

model=RandomForestClassifier()
model.fit(x_train, y_train)

y_predict=model.predict(x_test)

score=accuracy_score(y_predict,y_test)

print("Accuracy of prediction is {}%".format(score*100))


In [ ]:
f=open('model.p','wb')
pickle.dump({'model':model},f)
f.close()

# Test Classifier

In [ ]:
cap=cv2.VideoCapture(0)

model_dict=pickle.load(open('model.p','rb'))
model=model_dict['model']

mp_hands=mp.solutions.hands
mp_drawing=mp.solutions.drawing_utils
mp_drawing_styles=mp.solutions.drawing_styles

hands=mp_hands.Hands(static_image_mode=True,min_detection_confidence=0.3)

labels_dict={0:'A',1:'B',2:'L'}

while True:
    
    data_frame=[]
    x_=[]
    y_=[]
    
    ret,frame=cap.read()
    
    H,W,C=frame.shape
    
    frame_rgb=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    
    results=hands.process(frame_rgb)
    
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame, #image to draw
                hand_landmarks, #model output
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())
            
        for hand_landmarks in results.multi_hand_landmarks:
            for i in range(len(hand_landmarks.landmark)):
                x=hand_landmarks.landmark[i].x
                y=hand_landmarks.landmark[i].y
                data_frame.append(x)
                data_frame.append(y)
                x_.append(x)
                y_.append(y)
        
#       these coordinates are used to find the corners of the rectangle
        x1=int(min(x_)*W)-10
        y1=int(min(y_)*H)-10
        
        x2=int(max(x_)*W)-10
        y2=int(max(y_)*H)-10
                
        prediction=model.predict([np.asarray(data_frame)][:42])
        
        predicted_char=labels_dict[int(prediction[0])]
        
        cv2.rectangle(frame,(x1,y1),(x2,y2),(0,0,0),4)
        cv2.putText(frame, predicted_char, (x1,y1-10), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 0, 0), 3,
                    cv2.LINE_AA)
        
#         print(predicted_char)
    
    cv2.imshow('frame',frame)
    cv2.waitKey(1)
    
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)